In [8]:
import os
import pcapy as p
from scapy.all import *
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import struct


In [9]:
data = "capture.pcapng"

In [10]:
packets = rdpcap(data)
Config = []
IMGS = []
IMGSsmol = []
idxImags = []
IMGS2 = []
IMGSsmol2 = []
idxImags2 = []
start = 0
ix = 1
for k in packets:
    k = k[Raw].load
    if k[8] == 0x53: # URB submit
        if k[9] == 0x02: # URB submit
            TYPE = "URB_CONTROL"
        elif k[9] == 0x03: # BULK
            TYPE = "BULK"
        else:
            TYPE = "OTHER"
        EP = k[10] # EP
        if EP == 0:
            SENS ="OUT"
        else:
            SENS = "IN"
        if TYPE == "BULK":
            sizeRequested = k[33]*256+k[32] 
            DIC = {'ID': ix,"Sens":SENS,"TYPE":TYPE,"EP":k[10],"sizeRequested":sizeRequested,"LEN":len(k)}
        else:
            bmRequestType = k[40]
            bmRequest = k[41]
            bmValue  =  256*k[43]+k[42] 
            bmIndex  = k[44]+256*k[45]  
            bmLength =  k[46]+256*k[47]  

            #print(ix,hex(bmRequestType),bmRequest,hex(bmValue),bmIndex,bmLength,TYPE,SENS,hex(EP))
            DIC = {'ID': ix,"Sens":SENS,"TYPE":TYPE,"EP":k[10],"bmRequestType":bmRequestType,
                           "bmRequest":bmRequest,"bmValue":bmValue,"bmIndex":bmIndex,"bmLength":bmLength,"LEN":len(k)
                   }
        if len(k) > 64:
            DIC["payload"] = k[64:]
        Config.append(DIC)
            
    if k[8] == 0x43: # URB receive
        if 0:
            if (len(k) == 65):
                print(ix,hex(k[-1]))
            if (len(k) == 66):
                print(ix,hex(k[-1]+256*k[-2]  ))
                
    if (k[8] == 0x43) & ((k[10] == 0x86)): # URB receive  
        if len(k) > 20000:
            IMGS.append(k[64:])
            #print(ix)
            idxImags.append(str(ix)+"full")
        else:
            IMGSsmol.append(k[64:])
            idxImags.append(str(ix)+"short")
    if (k[8] == 0x43) & ((k[10] == 0x82)): # URB receive  
        if len(k) > 20000:
            IMGS2.append(k[64:])
            #print(ix)
            idxImags2.append(str(ix)+"full")
        else:
            IMGSsmol2.append(k[64:])
            idxImags2.append(str(ix)+"short")   
    ix +=1

In [11]:
df = pd.DataFrame(Config) 
DF = df.drop_duplicates(subset=["bmRequestType","bmRequest","bmValue","bmIndex","bmLength","sizeRequested","payload"])#.sort_values(by=["bmRequest","bmValue"])
print( len(df),len(DF) )
DF.to_pickle("conf.pkl")
DF

158 41


,ID,Sens,TYPE,EP,bmRequestType,bmRequest,bmValue,bmIndex,bmLength,LEN,payload,sizeRequested
0,2,IN,OTHER,129,0.0,0.0,0.0,0.0,0.0,64,NaN,NaN
1,3,IN,URB_CONTROL,128,163.0,0.0,0.0,1.0,4.0,64,NaN,NaN
2,5,OUT,URB_CONTROL,0,35.0,1.0,16.0,1.0,0.0,64,NaN,NaN
3,7,OUT,URB_CONTROL,0,35.0,1.0,25.0,1.0,0.0,64,NaN,NaN
4,9,OUT,URB_CONTROL,0,35.0,3.0,28.0,1.0,0.0,64,NaN,NaN
8,17,OUT,URB_CONTROL,0,35.0,1.0,20.0,1.0,0.0,64,NaN,NaN
9,19,OUT,URB_CONTROL,0,35.0,1.0,29.0,1.0,0.0,64,NaN,NaN
27,55,OUT,URB_CONTROL,0,35.0,3.0,27.0,1793.0,0.0,64,NaN,NaN
28,57,OUT,URB_CONTROL,0,35.0,3.0,27.0,1794.0,0.0,64,NaN,NaN
29,59,OUT,URB_CONTROL,0,35.0,3.0,27.0,1795.0,0.0,64,NaN,NaN


# Replays

In [1]:
#USB
import usb.core
import usb.util 
# Figures
import pandas as pd
import numpy as np
# Utilities
import json, sys
import base64
import struct

In [ ]:
device = usb.core.find(idVendor=0x0403, idProduct=0x601e)
    # initialize device
#usb.util.dispose_resources(device)

for config in device:  
    # The device was getting "Err 16 busy" on my ubuntu
    for i in range(config.bNumInterfaces):
        if device.is_kernel_driver_active(i):
            device.detach_kernel_driver(i) 
device.reset()

try:
    device.set_configuration()
except:
    print("Already connected")
                
try:
    device.set_configuration()
    device.reset()
except usb.core.USBError as e:
    sys.exit("Cannot set configuration the device: %s" % str(e))
    
# find first USBTMC interface
IFS = []
for cfg in device:
    for iface in cfg:
        #print(iface)
        IFS.append(iface)

device.set_configuration(cfg)
EP = []
# claim interface
for iface in IFS:
    print(" IFACE-> ")
    usb.util.claim_interface(device, iface)

    # don't need to set altsetting - USBTMC devices have 1 altsetting as per the spec

    
    for ep in iface:
        ep_dir = usb.util.endpoint_direction(ep.bEndpointAddress)
        ep_type = usb.util.endpoint_type(ep.bmAttributes)
        EP.append(ep)
        print(ep)

In [ ]:
DF["payload"] = DF.payload.fillna(-1)
for index, row in DF.iterrows():
    nop = [161,167]
    if  (row["EP"] == 0) & (row["ID"] > 131) & (not(row["ID"] in nop )):
        bmRT = int(row["bmRequestType"])
        bmR = int(row["bmRequest"])
        wV = int(row["bmValue"])
        wI = int(row["bmIndex"])
        
        if row["payload"] == -1: 
            PL = int(row["bmLength"])
        else: 
            PL = row["payload"]
            
        print("#",row["ID"]," ",bmRT,bmR,wV,wI,PL)
        m = device.ctrl_transfer(bmRequestType= bmRT,bRequest= bmR, wValue= wV, wIndex= wI , data_or_wLength=  PL)
        print(m)
print("Done")




In [14]:
nop = [189,191]
for BO in nop:
    row = DF[DF.ID == BO].iloc[0] 
    pp = EP[0].write(row["payload"])

    print("#",row["ID"],pp) 
print("Done")


# 189 20
# 191 20
Done


In [17]:
EP[-1].read(8*4096)

USBTimeoutError: [Errno 110] Operation timed out